In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import scipy.sparse as sp

import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from dgl.dataloading import GraphDataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from time import time
from tqdm import tqdm
from xgboost import XGBClassifier

from functions_HGPLS import train, test

In [3]:
dataset = dgl.data.FakeNewsDataset('gossipcop', feature_name='content')

In [69]:
dataset = dgl.data.GINDataset("PROTEINS", self_loop=False)

In [4]:
def create_features(g):
    degree = (g.out_degrees().float() + g.in_degrees().float()).numpy()
    num_edges = g.number_of_edges()
    num_nodes = g.number_of_nodes()
    nx_g = g.to_networkx().to_undirected()
    laplacian = nx.laplacian_matrix(nx_g).astype(np.float32).toarray()
    eigenvals, eigenvecs = np.linalg.eigh(laplacian)
    return [np.mean(degree), np.max(degree)/len(degree), 100*num_edges / (num_nodes * (num_nodes - 1))] + list(eigenvals[-3:])

In [71]:
dataset[0][0]

Graph(num_nodes=42, num_edges=162,
      ndata_schemes={'label': Scheme(shape=(), dtype=torch.int64), 'attr': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})

In [5]:
new_dataset = list()
labels = list()
for i in tqdm(range(len(dataset))):
    g, l = dataset[i]
    # features = list(np.mean(dataset.feature[g.ndata["_ID"]].numpy(), axis=0))
    # features = list(np.mean(g.ndata["attr"].numpy(), axis=0))
    features = create_features(g)
    new_dataset.append(features)
    labels.append(l)

100%|██████████| 5464/5464 [00:30<00:00, 179.57it/s]


In [6]:
train_dataset, test_dataset, train_labels, test_labels = train_test_split(new_dataset, labels, test_size=0.25, random_state=42)

In [7]:
xgb =XGBClassifier(random_state=0)

xgb.fit(train_dataset, train_labels)
predxgb = xgb.predict(test_dataset)

In [8]:
accuracy_score(test_labels, predxgb)

0.808199121522694